In [32]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

from tensorflow.keras.constraints import max_norm
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import activations
import numpy as np
import matplotlib.pyplot as plt

In [26]:
import tensorflow_datasets as tfds
(ds_train, ds_test) = tfds.load('mnist', split=['train', 'test'], shuffle_files=True,as_supervised=False)
X_data = tf.convert_to_tensor([i['image'] for i in tfds.as_numpy(ds_train)], dtype=tf.float32) / 255
X_data_test = tf.convert_to_tensor([i['image'] for i in tfds.as_numpy(ds_test)], dtype=tf.float32) / 255


In [134]:
def cov2d_block(x, filters=128, act=activations.relu, kern=(5, 5), strd=(1, 1), pad='same', BN=False, DO=0.):
    initializer = keras.initializers.RandomNormal(stddev=0.01)
    x = keras.layers.Conv2D(filters, kern, strd, pad, activation=act, use_bias=True, kernel_initializer=initializer)(x)
    if BN:
        x = keras.layers.BatchNormalization()(x)
    x = act(x)
    if DO > 0.:
        x = keras.layers.Dropout(D)(x)
    return x

def mp_conv2d_block(x, filters=128, act=activations.relu, kern=(5, 5), strd=(1, 1), pad='same', DO=1.):
    
    initializer = keras.initializers.RandomNormal(stddev=0.01)
    x = keras.layers.Conv2D(filters, kern, strd, pad, activation=act, use_bias=True, kernel_initializer=initializer)(x)
    x = act(x)
    x = keras.layers.Dropout(DO)(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    
    return x

def deconv2d_block(x, filters=64, act=activations.relu, kern=(5, 5), strd=(1, 1), pad='same', BN=False, DO=0.):
    initializer = RandomNormal(stddev=0.01)
    
    x = keras.layers.Conv2DTranspose(filters, kern, strd, pad, activation=act, use_bias=True, 
                                         kernel_initializer=initializer)(x)
    if BN:
        x = keras.layers.BatchNormalization()(x)
    x = act(x)
    if DO > 0.:
        x = keras.layers.Dropout(DO)(x)
    return x

def dense_block(x, input_dim=256, return_shape=256):
    initializer = RandomNormal(stddev=0.01)
    
    x = keras.layers.Dense(return_shape, activation='relu',kernel_initializer=initializer, use_bias=True)(x)
    return x


In [203]:
class Discriminator:
    def __init__(self, image_shape=(28, 28, 1)):
        self.image_shape = image_shape
        self.model_layer = []
        self.model = None
        
    def initialize_model(self):
        input_dims = keras.layers.Input(shape=self.image_shape)
        xk = mp_conv2d_block(input_dims, filters=128, act=activations.relu, kern=(5, 5), strd=(2, 2), pad='same', DO=1.)
        xk = mp_conv2d_block(xk, filters=256, act=activations.relu, kern=(5, 5), strd=(2, 2), pad='same', DO=1.)
        xk = mp_conv2d_block(xk, filters=256, act=activations.relu, kern=(5, 5), strd=(1, 1), pad='same', DO=1.)
        xk = keras.layers.Flatten()(xk)
        xk = keras.layers.Dense(1)(xk)
        
        self.model = keras.Model(input_dims, xk)
        
    def forward_model(self, input_tensor):
        return self.model(input_tensor)
    
    def extent_model(self):
        raise NotImplementedError
    
class Generator:
    def __init__(self, latent_z=128, out_latent=128, image_shape=(28, 28, 1)):
        self.out_latent = out_latent
        self.latent_z = latent_z
        self.image_shape = image_shape
        self.model_layer = []
        self.model = None 

    def initialize_model(self):
        input_dims = keras.layers.Input(shape=self.latent_z)
        xk = dense_block(input_dims, input_dim=self.latent_z, return_shape=self.out_latent)
        xk = keras.layers.Reshape((1, 1, self.out_latent))(xk)
        xk = deconv2d_block(xk, filters=128, act=activations.relu, kern=(4, 4), strd=(1, 1), 
                                pad='same', BN=False, DO=0.)
        xk = deconv2d_block(xk, filters=256, act=activations.relu, kern=(4, 4), strd=(1, 1), 
                                pad='same', BN=False, DO=0.)
        xk = deconv2d_block(xk, filters=256, act=activations.relu, kern=(4, 4), strd=(1, 1), 
                                pad='same', BN=False, DO=0.)
        xk = keras.layers.Flatten()(xk)
        xk = keras.layers.Dense(tf.reduce_prod(self.image_shape))(xk)
        xk = keras.layers.Reshape(self.image_shape)(xk)
        self.model = keras.Model(input_dims, xk)
        
    def forward_model(self, input_tensor):
        return self.model(input_tensor)
    
    def extent_model(self):
        raise NotImplementedError


In [204]:
T_disc = Discriminator()
T_disc.initialize_model()

T_gen = Generator()
T_gen.initialize_model()


In [251]:
def grad_loss_0(D, alpha, p_r, p_g):
    interpolate = p_r + alpha * (p_g - p_r)
    
    with tf.name_scope("Gradient_Penalty"):
        with tf.GradientTape() as g_tape:
            g_tape.watch(interpolate)
            inter_pred_disc = D.forward_model(interpolate)
        g_grad = g_tape.gradient(inter_pred_disc, [interpolate])
        norm_g = tf.sqrt(tf.reduce_sum(tf.square(g_grad)))
        g_grad = tf.reduce_mean(tf.math.squared_difference(norm_g, 1))
    
    return g_grad
        
def generator_loss_0(z, G, D, train_sets):
    fake_img = G.forward_model(z)
    real_score = D.forward_model(fake_img)
    fake_score = D.forward_model(train_sets)
    gen_loss = - tf.nn.softplus(real_score) 
    
    return gen_loss

def discrim_loss_0(z, G, D, train_sets, g_penalty=0.):
    fake_img = G.forward_model(z)
    real_score = D.forward_model(fake_img)
    fake_score = D.forward_model(train_sets)
    disc_loss = tf.nn.softplus(fake_score) + tf.nn.softplus(- real_score) 
    
    if g_penalty > 0.:
        grad_loss_dc = grad_loss_0(D, alpha=0.5, p_r=train_sets, p_g=fake_img)
    disc_loss += grad_loss_dc
    return disc_loss, grad_loss_dc

In [252]:
z_k = np.random.rand(10, 128)
with tf.GradientTape() as gen_tp:
    gen_loss = generator_loss_0(z=z_k, G=T_gen, D=T_disc, train_sets=X_data[:10])
gen_grad = gen_tp.gradient(gen_loss, T_gen.model.trainable_variables)

In [269]:
z_k = np.random.rand(10, 128)
with tf.GradientTape() as disc_tp:
    disc_loss, g_ld = discrim_loss_0(z=z_k, G=T_gen, D=T_disc, train_sets=X_data[:10], g_penalty=1.)
disc_grad = disc_tp.gradient(disc_loss, T_disc.model.trainable_variables)

In [278]:
def loop_train(epoch=50):
    
    for k in range(epoch):
        
        pass